### **HuggingFace Embeddings**

In [ ]:
texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

**참고(Reference)**

![](./images/top-ranked-embeddings.png)

- [(출처) Kor-IR: 한국어 검색을 위한 임베딩 벤치마크](https://github.com/teddylee777/Kor-IR?tab=readme-ov-file)

### **HuggingFace Endpoint Embedding**

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
import os 

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_TOKEN'),
)

In [ ]:
# ebmed_documents() 함수로 임베딩
embedded_documents = hf_embeddings.embed_documents(texts)

c:\Users\rmsgh\anaconda3\envs\nlp\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


[[0.05119204893708229,
  0.011308944784104824,
  0.01282633375376463,
  -0.03129679709672928,
  0.02438686601817608,
  -0.03183966875076294,
  -0.04137240722775459,
  0.0072222305461764336,
  0.04447772353887558,
  -0.023085806518793106,
  0.028797708451747894,
  0.01078395638614893,
  -0.013323909603059292,
  -0.028462322428822517,
  -0.04694923013448715,
  -0.021683210507035255,
  -0.025736214593052864,
  0.0019589660223573446,
  0.0007779102888889611,
  -0.011368456296622753,
  0.026559386402368546,
  0.0023800611961632967,
  -0.006590425502508879,
  -0.050234053283929825,
  -0.030206531286239624,
  -0.003619129303842783,
  -0.05006798356771469,
  -0.015691451728343964,
  -0.01871846616268158,
  -0.026770109310746193,
  0.027322892099618912,
  0.0075068180449306965,
  -0.03852769732475281,
  -0.040268827229738235,
  -0.012338823638856411,
  0.023780297487974167,
  0.023508688434958458,
  0.029794255271553993,
  -0.01239994540810585,
  0.06362662464380264,
  -0.01546317059546709,
  0

In [5]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

[HuggingFace Endpoint Embedding]
Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


In [6]:
# Document Embedding 수행
embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")

In [7]:
len(embedded_query)

1024

### 유사도 계산

**벡터 내적을 통한 유사도 계산**
- 벡터 내적(dot product)을 사용하여 유사도를 계산 

- 유사도 계산 공식:

$$ \text{similarities} = \mathbf{query} \cdot \mathbf{documents}^T $$

#### 벡터 내적의 수학적 의미

**벡터 내적 정의**

벡터 $\mathbf{a}$와 $\mathbf{b}$의 내적은 다음과 같이 정의됩니다:
$$ \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i $$

**코사인 유사도와의 관계**

벡터 내적은 다음과 같은 성질을 가집니다.
$$ \mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos \theta $$

여기서,
- $\|\mathbf{a}\|$와 $\|\mathbf{b}\|$는 각각 벡터 $\mathbf{a}$와 $\mathbf{b}$의 크기(노름, Euclidean norm)입니다.
- $\theta$는 두 벡터 사이의 각도입니다.
- $\cos \theta$는 두 벡터 사이의 코사인 유사도입니다.

**벡터 내적의 유사도 해석**
내적 값이 클수록 (양의 큰 값일수록),
- 두 벡터의 크기($\|\mathbf{a}\|$와 $\|\mathbf{b}\|$)가 크고,
- 두 벡터 사이의 각도($\theta$)가 작으며 ($\cos \theta$가 1에 가까움),

이는 두 벡터가 유사한 방향을 가리키고, 크기가 클수록 더 유사하다는 것을 의미

**벡터의 크기(노름) 계산**

Euclidean norm 정의
벡터 $\mathbf{a} = [a_1, a_2, \ldots, a_n]$에 대해, Euclidean norm $\|\mathbf{a}\|$는 다음과 같이 정의됩니다:
$$ \|\mathbf{a}\| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} $$

----

query 와 embedding_document 간의 유사도 계산

In [12]:
import numpy as np

# 내적을 통한 유사도 계산
result = np.array(embedded_query) @ np.array(embedded_documents).T

result[result.argmax()]

0.8956487815512959

In [13]:
sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]
sorted_idx

array([3, 1, 2, 0, 4], dtype=int64)

In [14]:
print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

[Query] LangChain 에 대해서 알려주세요.
[0] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[1] LangChain simplifies the process of building applications with large language models

[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[3] 안녕, 만나서 반가워.

[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.



### **HuggingFace Embeddings**



### `intfloat/multilingual-e5-large-instruct`

- [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct)
- [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"
# model_name = "intfloat/multilingual-e5-large"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "mps"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
%time
# Document
embedded_documents1 = hf_embeddings.embed_documents(texts)

In [ ]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

## BGE-M3 임베딩

아래의 옵션 중 에러가 발생할 수 있는 옵션 설명

- `{"device": "mps"}`: GPU 대신 MPS를 사용하여 임베딩 계산을 수행 (Mac 사용자)
- `{"device": "cuda"}`: GPU를 사용하여 임베딩 계산을 수행 (Linux, Windows 사용자, 단 CUDA 설치 필요)
- `{"device": "cpu"}`: CPU를 사용하여 임베딩 계산을 수행 (모든 사용자

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "mps"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [ ]:
%time
# Document
embedded_documents = hf_embeddings.embed_documents(texts)

In [ ]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

In [ ]:
import numpy as np

embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")
embedded_documents = hf_embeddings.embed_documents(texts)

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

### `FlagEmbedding` 을 활용하는 방식

**참고**
- [FlagEmbedding - BGE-M3 Usage](https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/BGE_M3#usage)


`FlagEmbedding` 에서 제공하는 세 가지 접근법을 조합하면, 더욱 강력한 검색 시스템을 구축

- Dense Vector: BGE-M3의 다국어, 다중 작업 능력을 기반으로 함
- Lexical weight를 활용한 sparse embedding으로 정확한 단어 매칭을 수행
- ColBERT의 multi-vector 접근법으로 문맥을 고려한 세밀한 매칭 수행

In [ ]:
# FlagEmbedding 설치
!pip install -qU FlagEmbedding

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"
bge_embeddings = BGEM3FlagModel(
    model_name, use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.

bge_embedded = bge_embeddings.encode(
    texts,
    batch_size=12,
    max_length=8192,  # 이렇게 긴 길이가 필요하지 않은 경우 더 작은 값을 설정하여 인코딩 프로세스의 속도를 높일 수 
)["dense_vecs"]

In [ ]:
bge_embedded.shape

In [ ]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

In [ ]:
from FlagEmbedding import BGEM3FlagModel

bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_dense=True)

In [ ]:
# 결과 출력(행, 열)
bge_encoded["dense_vecs"].shape

### Sparse Embedding (Lexical Weight)

Sparse embedding은 벡터의 대부분의 값이 0인 고차원 벡터를 사용하는 임베딩 방식입니다. Lexical weight를 활용한 방식은 단어의 중요도를 고려하여 임베딩을 생성

**작동 방식**
1. 각 단어에 대해 lexical weight(어휘적 가중치)를 계산 이는 TF-IDF나 BM25 같은 방법을 사용할 수 
2. 문서나 쿼리의 각 단어에 대해, 해당 단어의 lexical weight를 사용하여 sparse vector의 해당 차원에 값을 할당
3. 결과적으로 문서나 쿼리는 대부분의 값이 0인 고차원 벡터로 표현됩니다.

**장점**
- 단어의 중요도를 직접적으로 반영할 수 
- 특정 단어나 구문을 정확히 매칭할 수 
- 계산이 상대적으로 빠릅니다.

In [ ]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_sparse=True)

In [ ]:
lexical_scores1 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][0]
)
lexical_scores2 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][1]
)
# 0 <-> 0
print(lexical_scores1)
# 0 <-> 1
print(lexical_scores2)

### Multi-Vector (ColBERT)

ColBERT(Contextualized Late Interaction over BERT)는 문서 검색을 위한 효율적인 방법입니다. 이 방식은 문서와 쿼리를 여러 개의 벡터로 표현하는 multi-vector 접근법을 사용합니다.

**작동 방식**

1. 문서의 각 토큰에 대해 별도의 벡터를 생성합니다. 즉, 하나의 문서는 여러 개의 벡터로 표현됩니다.
2. 쿼리도 마찬가지로 각 토큰에 대해 별도의 벡터를 생성합니다.
3. 검색 시, 쿼리의 각 토큰 벡터와 문서의 모든 토큰 벡터 사이의 유사도를 계산
4. 이 유사도들을 종합하여 최종 검색 점수를 계산합니다.

**장점**
- 토큰 수준의 세밀한 매칭이 가능합니다.
- 문맥을 고려한 임베딩을 생성할 수 
- 긴 문서에 대해서도 효과적으로 작동합니다.

In [ ]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_colbert_vecs=True)

In [ ]:
colbert_scores1 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][0]
)
colbert_scores2 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][1]
)
# 0 <-> 0
print(colbert_scores1)
# 0 <-> 1
print(colbert_scores2)